# Import

In [1]:
from collections import Counter
import re

import numpy as np
import pandas as pd

from db.mongo import MyMongo

# DB

In [2]:
with MyMongo() as db:
    cvs = db.get_df_from_table('cvs', 'cvs')
    distance = db.get_df_from_table('cvs', 'src_dst')

stop_words = ['ccup', 'mercury 전자담배', 'in cups', 'focus24', 'lgstar', 'gstop', 'sicuro', 'gs슈퍼', 'gs칼텍스', 't-cube']
stop_string = '|'.join(stop_words)

cvs = cvs.loc[~cvs['사업장명'].str.contains(stop_string, flags=re.IGNORECASE, regex=True)]


idx_tmp1 = distance['src_name'].str.contains(stop_string, flags=re.IGNORECASE, regex=True)
idx_tmp2 = distance['dst_name'].str.contains(stop_string, flags=re.IGNORECASE, regex=True)

# print(len(distance))
# print(len(distance.loc[~(idx_tmp1 | idx_tmp2), cols_distance]))

distance = distance.loc[~(idx_tmp1 | idx_tmp2)]
del idx_tmp1, idx_tmp2

<--Mongo Connected.
Mongo Connection Closed.-->


# cvs 정상영업: 39484

In [3]:
print(len(cvs.loc[cvs['상세영업상태명']=='정상영업']))

41252


# Index: naver_distance != NaN

In [4]:
distance.head(1)
idx_non_naver = distance['naver_distance'].isna()
print(len(distance.loc[idx_non_naver]))
distance.loc[idx_non_naver].head(1)

3116


,_id,dst,dst_lat,dst_lng,dst_name,line_distance,naver_distance,src,src_lat,src_lng,src_name
218,5c0d43659085ca6d107fa2d0,2003349001205600040,37.572488,126.974268,미니스톱 문화회관점,0.069688,NaN,2016300016905600038,37.572487,126.974268,미니스톱 세종문화회관점


# Columns

In [5]:
cvs.head(1)
cvs.columns
cols_cvs = ['사업장명', '관리번호', 'lat', 'lng', '상세영업상태명', '도로명전체주소', '소재지전체주소', 'type', '지역']
cvs.loc[::, cols_cvs].head(1)
# print(distance.columns)
cols_distance = ['src_name', 'src', 'src_status', 'src_type', 'src_lat', 'src_lng', 'dst_name', 'dst', 'dst_status', 'dst_type', 'dst_lat',
                 'dst_lng', 'line_distance', 'naver_distance']

/home/jake/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


# Add 'status' to 'distance'

In [6]:
tmp_distance = distance.merge(cvs[['관리번호', '상세영업상태명']], left_on='src', right_on='관리번호', how='left'); tmp_distance.head(1)
tmp_distance = tmp_distance.rename(columns={'상세영업상태명': 'src_status'}); tmp_distance.head(1)
tmp_distance = tmp_distance.drop(columns=['관리번호'])

tmp_distance = tmp_distance.merge(cvs[['관리번호', '상세영업상태명']], left_on='dst', right_on='관리번호', how='left'); tmp_distance.head(1)
tmp_distance = tmp_distance.rename(columns={'상세영업상태명': 'dst_status'}); tmp_distance.head(1)
distance_status = tmp_distance.drop(columns=['관리번호'])
del tmp_distance
distance_status.loc[::, cols_distance].head(1)

/home/jake/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,src_name,src,src_status,src_type,src_lat,src_lng,dst_name,dst,dst_status,dst_type,dst_lat,dst_lng,line_distance,naver_distance
0,GS25동숭점,1999300007605600150,정상영업,NaN,37.583669,127.002563,세븐일레븐 종로대학로점,2009300012905600010,폐업처리,NaN,37.583248,127.002445,47.934733,44.0


# Add: 'type' to 'distance'

In [7]:
# types = ['gs', '지에스', '씨유', 'cu', '세븐일레븐', '코리아세븐', '이마트24', 'emart24', '미니스톱', '씨스페이스', '씨스페이시스']
# normalised_types = {
#     'gs': 'GS25',
#     '지에스': 'GS25',
#     '씨유': 'CU',
#     'cu': 'CU',
#     '세븐일레븐': '세븐일레븐',
#     '코리아세븐': '세븐일레븐',
#     '이마트24': '이마트24',
#     'emart24': '이마트24',
#     '미니스톱': '미니스톱',
#     '씨스페이스': '씨스페이스',
#     '씨스페이시스': '씨스페이스',
# }

types = [
    'gs', '지에스', 'lg25', '엘지25',
     '씨유', 'cu', '훼미리마트', '훼밀리마트', '패밀리마트', 'familymart',
     '세븐일레븐', '코리아세븐', '바이더웨이', 'buytheway',
     '이마트24', 'emart24', '위드미',
     '미니스톱', '미니스탑', '대상유통',
     '씨스페이스', '씨스페이시스'
]

normalised_types = {
    'gs': 'GS25',
    '지에스': 'GS25',
    'lg25': 'GS25',
    '엘지25': 'GS25',
    
    '씨유': 'CU',
    'cu': 'CU',
    '훼미리마트': 'CU',
    '훼밀리마트': 'CU',
    '패밀리마트': 'CU',
    'familymart': 'CU',
    
    '세븐일레븐': '세븐일레븐',
    '코리아세븐': '세븐일레븐',
    '바이더웨이': '세븐일레븐',
    'buytheway': '세븐일레븐',
    
    '이마트24': '이마트24',
    'emart24': '이마트24',
    '위드미': '이마트24',
    
    '미니스톱': '미니스톱',
    '미니스탑': '미니스톱',
    '대상유통': '미니스톱',
    
    '씨스페이스': '씨스페이스',
    '씨스페이시스': '씨스페이스',
}

for idx, row in distance_status.loc[::, cols_distance].iterrows():
    src_name = row['src_name'].lower()
    dst_name = row['dst_name'].lower()

    for t in types:
        if t in src_name:
            distance_status.at[idx, 'src_type'] = normalised_types[t]
        if t in dst_name:
            distance_status.at[idx, 'dst_type'] = normalised_types[t]

for idx, row in cvs.loc[::, cols_cvs].iterrows():
    name = row['사업장명'].lower()
    
    for t in types:
        if t in name:
            cvs.at[idx, 'type'] = normalised_types[t]

# print(len(distance))
print(len(distance_status.loc[distance_status['src_type'].isna()]))
print(len(distance_status.loc[distance_status['dst_type'].isna()]))
print(len(cvs.loc[cvs['type'].isna()]))

distance_status.loc[::, cols_distance].head(5)
# cvs.loc[idx_closed&~idx_non_closed_date, cols_cvs].head(5)

/home/jake/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


0
0
0


,src_name,src,src_status,src_type,src_lat,src_lng,dst_name,dst,dst_status,dst_type,dst_lat,dst_lng,line_distance,naver_distance
0,GS25동숭점,1999300007605600150,정상영업,GS25,37.583669,127.002563,세븐일레븐 종로대학로점,2009300012905600010,폐업처리,세븐일레븐,37.583248,127.002445,47.934733,44.0
1,이마트24 광화문시대점,2007300010105600043,정상영업,이마트24,37.573773,126.973126,GS25 종로내수,2013300012905600007,정상영업,GS25,37.573431,126.973271,40.155229,124.0
2,이마트24 광화문시대점,2007300010105600043,정상영업,이마트24,37.573773,126.973126,GS25 종로용비점,2006300007605600008,폐업처리,GS25,37.573431,126.973271,40.155229,124.0
3,이마트24 광화문시대점,2007300010105600043,정상영업,이마트24,37.573773,126.973126,GS25 종로용비점,2005300007605600042,폐업처리,GS25,37.573431,126.973271,40.155229,124.0
4,이마트24 광화문시대점,2007300010105600043,정상영업,이마트24,37.573773,126.973126,GS25 종로용비점,2010300012905600016,폐업처리,GS25,37.573431,126.973271,40.155229,124.0


# Validation: Combination Sequence

In [8]:
a, b = '2010300012905600060', '2007300010105600014'
idx_sequence = ((distance_status['src']==a)&(distance_status['dst']==b))\
                | ((distance_status['dst']==a)&(distance_status['src']==b))
distance_status.loc[idx_sequence, cols_distance]
# distance_status.loc[idx_active & idx_sequence, cols_distance]


,src_name,src,src_status,src_type,src_lat,src_lng,dst_name,dst,dst_status,dst_type,dst_lat,dst_lng,line_distance,naver_distance
26,세븐일레븐 종로두산점,2007300010105600014,정상영업,세븐일레븐,37.572532,126.981793,씨유 종로공평점,2010300012905600060,폐업처리,CU,37.571915,126.982712,106.105078,135.0
232,씨유 종로공평점,2010300012905600060,폐업처리,CU,37.571915,126.982712,세븐일레븐 종로두산점,2007300010105600014,정상영업,세븐일레븐,37.572532,126.981793,106.105078,142.0


# Dedupe: 13197 -> 13209

In [9]:
deduped = distance_status[pd.DataFrame(np.sort(distance_status[['src','dst']].values,1)).duplicated()]

len(deduped)

23413

In [10]:
13209 - 13197

12

# Index: Active (1809->1810)

In [11]:
idx_active = (deduped['src_status']=='정상영업') & (deduped['dst_status']=='정상영업')
print(len(deduped.loc[idx_active, cols_distance]))
deduped.loc[idx_active, cols_distance].head(5)

2604


,src_name,src,src_status,src_type,src_lat,src_lng,dst_name,dst,dst_status,dst_type,dst_lat,dst_lng,line_distance,naver_distance
149,세븐일레븐 종로으뜸점,2014300016905600057,정상영업,세븐일레븐,37.569433,126.990686,GS25국일관점,2012300012905600021,정상영업,GS25,37.569703,126.990025,65.557707,142.0
186,씨유 종로대로점,2018300016905600037,정상영업,CU,37.570053,126.989980,GS25국일관점,2012300012905600021,정상영업,GS25,37.569703,126.990025,39.076506,92.0
214,미니스톱 세종문화회관점,2016300016905600038,정상영업,미니스톱,37.572487,126.974268,지에스25종로국카사옥,2018300016905600049,정상영업,GS25,37.572735,126.973280,91.282711,226.0
230,씨유 종로공평점,2017300016905600098,정상영업,CU,37.571915,126.982712,세븐일레븐 종로두산점,2007300010105600014,정상영업,세븐일레븐,37.572532,126.981793,106.105078,142.0
236,씨유종로창신점,2015300016905600068,정상영업,CU,37.572565,127.014793,GS25 S6 동묘앞역점,2018300016905600041,정상영업,GS25,37.572200,127.015680,88.066993,146.0


# Validation: Deduped

In [12]:
a, b = '2007300010105600014', '2010300012905600060'
idx_sequence = ((deduped['src']==a)&(deduped['dst']==b))\
                | ((deduped['dst']==a)&(deduped['src']==b))
deduped.loc[idx_sequence, cols_distance]

# distance_status.loc[idx_active & idx_sequence, cols_distance]

# 이마트24 광화문시대점	2007300010105600043	정상영업	37.573773	126.973126	GS25 종로내수	2013300012905600007	
# 세븐일레븐 독립문역점	2007300012905600006	정상영업	37.574565	126.958124	씨유독립문점	2013300016905600012
# 세븐일레븐 종로배화점	2008300012905600082	정상영업	37.577164	126.969439	씨유 종로사직공원점	2013300012905600009
# 세븐일레븐 종로향기점	2003300007605600130	정상영업	37.574283	127.021653	GS25 숭인본점	1998300007605600254
# CU M강남역점	2018322016205600170	정상영업	37.498165	127.028308	미니스톱 강남시티3점	2013322016205600169	

# 2007300010105600014  2017300016905600098
# 2007300010105600014  2010300012905600060

,src_name,src,src_status,src_type,src_lat,src_lng,dst_name,dst,dst_status,dst_type,dst_lat,dst_lng,line_distance,naver_distance
232,씨유 종로공평점,2010300012905600060,폐업처리,CU,37.571915,126.982712,세븐일레븐 종로두산점,2007300010105600014,정상영업,세븐일레븐,37.572532,126.981793,106.105078,142.0


# Validation: 실제 지도와 비교

In [13]:
deduped.loc[idx_active, cols_distance].head(1)
tmp = deduped.loc[idx_active, cols_distance].groupby('src').agg({'src_name':'count'})
tmp.loc[tmp['src_name']>=3].sort_values('src_name', ascending=False)
# 2013322016205600169, 2009322012705600008, 2018322016205600170

,src_name
src,
2015322016205600127,6
2009322012705600008,6
2011567016105600052,5
2009474004705600026,5
2013322016205600169,5
2008322012705600092,4
2017356013705600132,4
2000321007605600078,4
2006322008305600177,4


In [14]:
deduped.loc[idx_active, cols_distance].head(5)

# 이마트24 광화문시대점	2007300010105600043	정상영업	37.573773	126.973126	GS25 종로내수	2013300012905600007	
# 세븐일레븐 독립문역점	2007300012905600006	정상영업	37.574565	126.958124	씨유독립문점	2013300016905600012
# 세븐일레븐 종로배화점	2008300012905600082	정상영업	37.577164	126.969439	씨유 종로사직공원점	2013300012905600009
# 세븐일레븐 종로향기점	2003300007605600130	정상영업	37.574283	127.021653	GS25 숭인본점	1998300007605600254
# CU M강남역점	2018322016205600170	정상영업	37.498165	127.028308	미니스톱 강남시티3점	2013322016205600169	

# 2007300010105600014  2017300016905600098
# 2007300010105600014  2010300012905600060

,src_name,src,src_status,src_type,src_lat,src_lng,dst_name,dst,dst_status,dst_type,dst_lat,dst_lng,line_distance,naver_distance
149,세븐일레븐 종로으뜸점,2014300016905600057,정상영업,세븐일레븐,37.569433,126.990686,GS25국일관점,2012300012905600021,정상영업,GS25,37.569703,126.990025,65.557707,142.0
186,씨유 종로대로점,2018300016905600037,정상영업,CU,37.570053,126.989980,GS25국일관점,2012300012905600021,정상영업,GS25,37.569703,126.990025,39.076506,92.0
214,미니스톱 세종문화회관점,2016300016905600038,정상영업,미니스톱,37.572487,126.974268,지에스25종로국카사옥,2018300016905600049,정상영업,GS25,37.572735,126.973280,91.282711,226.0
230,씨유 종로공평점,2017300016905600098,정상영업,CU,37.571915,126.982712,세븐일레븐 종로두산점,2007300010105600014,정상영업,세븐일레븐,37.572532,126.981793,106.105078,142.0
236,씨유종로창신점,2015300016905600068,정상영업,CU,37.572565,127.014793,GS25 S6 동묘앞역점,2018300016905600041,정상영업,GS25,37.572200,127.015680,88.066993,146.0


# ---

# damaged & active

## < 150m: 1220->1221->1362

In [15]:
idx_lt_150 = deduped['naver_distance'] < 150
lt_150 = deduped.loc[idx_active & idx_lt_150, cols_distance]
print(len(lt_150))

1737


## < 100m: 616->617->681

In [16]:
idx_lt_100 = deduped['naver_distance'] < 100
lt_100 = deduped.loc[idx_active & idx_lt_100, cols_distance]
print(len(lt_100))

871


## 근접 active 편의점 수: 2145->2147->2373

In [17]:
cvs_set = set()
cvs_set.update(lt_150['src'].tolist())
cvs_set.update(lt_150['dst'].tolist())

print(len(cvs_set))

3011


In [18]:
idx_damaged = cvs['관리번호'].isin(cvs_set)
print(len(cvs.loc[idx_damaged, cols_cvs]))
cvs.loc[idx_damaged, cols_cvs].head(1)

3011


/home/jake/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,type,지역
17,씨유 종로성대점,2010300012905600034,37.58443693473411,126.99702012247322,정상영업,서울특별시 종로구 성균관로5길 6 (명륜2가),서울특별시 종로구 명륜동2가 137번지 2호,CU,NaN


In [19]:
cvs_set_100 = set()
cvs_set_100.update(lt_100['src'].tolist())
cvs_set_100.update(lt_100['dst'].tolist())

print(len(cvs_set_100))

1610


In [20]:
idx_damaged_100 = cvs['관리번호'].isin(cvs_set_100)
print(len(cvs.loc[idx_damaged_100, cols_cvs]))
cvs.loc[idx_damaged_100, cols_cvs].head(1)

1610


/home/jake/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,type,지역
17,씨유 종로성대점,2010300012905600034,37.58443693473411,126.99702012247322,정상영업,서울특별시 종로구 성균관로5길 6 (명륜2가),서울특별시 종로구 명륜동2가 137번지 2호,CU,NaN


## Counter: 근접킹 편의점

In [21]:
c_near_active = Counter(lt_150['src'].tolist())
c_near_active.update(lt_150['dst'].tolist())
c_near_active.most_common()

[('2018322016205600170', 6),
 ('2002322016205600000', 6),
 ('2017308013905600027', 4),
 ('2001308007505600851', 4),
 ('2013322016205600169', 4),
 ('2008322012705600092', 4),
 ('2008396012205600083', 4),
 ('2017396012805600010', 4),
 ('2017396012805600007', 4),
 ('2015402010805600067', 4),
 ('2017402010805600059', 4),
 ('2016304019005600022', 4),
 ('2015321015505600027', 4),
 ('2016321015505600116', 4),
 ('2017313020105600060', 3),
 ('2018313020105600003', 3),
 ('2017313020105600028', 3),
 ('2009318011705600055', 3),
 ('2005321007605600138', 3),
 ('2018322016205600155', 3),
 ('2009322012705600008', 3),
 ('2010322016205600015', 3),
 ('2012333010305600085', 3),
 ('2016333013205600056', 3),
 ('2016354011905600004', 3),
 ('2017354011905600147', 3),
 ('2017370010405600017', 3),
 ('2013370007805600009', 3),
 ('2006387003705600167', 3),
 ('2013398010805600031', 3),
 ('2012402010805600043', 3),
 ('2005402008105600046', 3),
 ('2016566002705600068', 3),
 ('2015507017805600012', 3),
 ('20125070178

In [22]:
# c_near_active.keys()
df_counter = pd.DataFrame(data={'관리번호': list(c_near_active.keys()), '근접 편의점 수': list(c_near_active.values())})
print(len(cvs.loc[idx_damaged, cols_cvs].merge(df_counter, on='관리번호', how='inner')))

df_counter = df_counter.merge(cvs.loc[idx_damaged, cols_cvs], on='관리번호', how='left')
print(len(df_counter))
df_counter.head(1)

3011
3011


/home/jake/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,관리번호,근접 편의점 수,사업장명,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,type,지역
0,2014300016905600057,1,세븐일레븐 종로으뜸점,37.56943324692268,126.99068635514514,정상영업,서울특별시 종로구 수표로18길 19 (관수동),서울특별시 종로구 관수동 24번지,세븐일레븐,NaN


In [23]:
df_counter.sort_values('근접 편의점 수', ascending=False).head(10)

,관리번호,근접 편의점 수,사업장명,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,type,지역
1780,2018322016205600170,6,CU M강남역점,37.498164651039694,127.02830790088069,정상영업,"서울특별시 강남구 강남대로 지하 396, 강남역 222-103, 104호 (역삼동)","서울특별시 강남구 역삼동 858번지 강남역 222-103, 104호",CU,NaN
1781,2002322016205600000,6,GS25 강남메트로,37.498164651039694,127.02830790088069,정상영업,"서울특별시 강남구 강남대로 지하 396, 강남역 A-10, 11호 (역삼동)","서울특별시 강남구 역삼동 858번지 강남역, A-10, 11호",GS25,NaN
81,2017308013905600027,4,씨스페이스 수유시네마,37.635664815367456,127.02383724952873,정상영업,"서울특별시 강북구 도봉로 308, 10층 10-3호 (번동, 북한산스카이)",서울특별시 강북구 번동 449번지 1호,씨스페이스,NaN
1604,2016304019005600022,4,씨유 세종대후문점,37.55367205833859,127.07361072813396,정상영업,"서울특별시 광진구 군자로 127, 1층 (군자동)",NaN,CU,NaN
781,2017402010805600059,4,GS25 산본중앙점,37.359676629321655,126.9320470098119,정상영업,"경기도 군포시 산본로323번길 16-14, 1층 4호 (산본동)",경기도 군포시 산본동 1134번지 센타빌딩 1층 4호,GS25,NaN
780,2015402010805600067,4,세븐일레븐 산본희망점,37.35930604417471,126.9331670952039,정상영업,"경기도 군포시 산본로323번길 7, 101호 (산본동, 금성프라자빌딩)",NaN,세븐일레븐,NaN
229,2013322016205600169,4,미니스톱 강남시티3점,37.49857910524426,127.02834424532402,정상영업,"서울특별시 강남구 테헤란로 103, 11,12호 (역삼동)","서울특별시 강남구 역삼1동 822번지 11,12호",미니스톱,NaN
256,2008322012705600092,4,세븐일레븐대치나인점,37.50398835114941,127.05098351139713,정상영업,"서울특별시 강남구 테헤란로64길 14 (대치동,나인오피스텔 101호)",서울특별시 강남구 대치동 889번지 56호 나인오피스텔 101호,세븐일레븐,NaN
1773,2016321015505600116,4,씨유 강남아크로텔점,37.4923152323998,127.02864945035205,정상영업,"서울특별시 서초구 효령로77길 34, 105,106호 (서초동)",서울특별시 서초구 서초동 1337번지 2호 105~6호,CU,NaN
1772,2015321015505600027,4,지에스25 서초세인트점,37.4923152323998,127.02864945035205,정상영업,"서울특별시 서초구 사임당로 174, 107호 (서초동, 강남미래타워)",서울특별시 서초구 서초동 1337번지,GS25,NaN


## Group '근접 active' by 'type'

In [24]:
tmp = cvs.loc[idx_damaged, cols_cvs].groupby('type').agg({'관리번호':'count'})
print(tmp.sum())
print(tmp.sort_values('관리번호', ascending=False))

del tmp
# cvs.loc[idx_damaged, cols_cvs].groupby('type').agg({'관리번호':'count'})

/home/jake/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


관리번호    3011
dtype: int64
       관리번호
type       
GS25   1005
CU      889
세븐일레븐   631
미니스톱    288
이마트24   178
씨스페이스    20


## Group 'active' by 'type'

In [25]:
tmp = cvs.loc[cvs['상세영업상태명']=='정상영업', cols_cvs].groupby('type').agg({'관리번호':'count'})
print(tmp.sum())
print(tmp.sort_values('관리번호', ascending=False))

del tmp
# cvs.loc[idx_damaged, cols_cvs].groupby('type').agg({'관리번호':'count'})

/home/jake/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


관리번호    41252
dtype: int64
        관리번호
type        
CU     12809
GS25   12807
세븐일레븐   9375
이마트24   3383
미니스톱    2661
씨스페이스    217


# 같은 회사 근접편의점 찾기

In [39]:
idx_same_corp = lt_150['src_type']==lt_150['dst_type']

print(len(lt_150.loc[idx_same_corp, cols_distance]))
lt_150.loc[idx_same_corp, ::].sort_values('naver_distance')

223


,src_name,src,src_status,src_type,src_lat,src_lng,dst_name,dst,dst_status,dst_type,dst_lat,dst_lng,line_distance,naver_distance
8613,미니스톱 역삼만남점,2010322016205600015,정상영업,미니스톱,37.495580,127.031844,미니스톱,2009399021405600024,정상영업,미니스톱,37.495578,127.031850,0.566976,12.0
8609,미니스톱 역삼만남점,2010322016205600015,정상영업,미니스톱,37.495580,127.031844,미니스톱,2007324013905600118,정상영업,미니스톱,37.495578,127.031850,0.566976,12.0
8611,미니스톱 역삼만남점,2010322016205600015,정상영업,미니스톱,37.495580,127.031844,미니스톱,1997362001205600154,정상영업,미니스톱,37.495578,127.031850,0.566976,12.0
64998,바이더웨이무역센터,2000322008305615906,정상영업,세븐일레븐,37.502014,127.057922,세븐일레븐 대치마루점,2009322012705600079,정상영업,세븐일레븐,37.502003,127.057917,1.266743,13.0
64645,훼미리마트성수점,200030300740560구16,정상영업,CU,37.544667,127.054900,씨유 성수점,2008303010305600097,정상영업,CU,37.544629,127.054673,20.421210,17.0
7744,지에스25(GS25)삼성메디슨점,2018322016205600155,정상영업,GS25,37.507257,127.065775,GS25편의점,2017338007105600012,정상영업,GS25,37.507252,127.065812,3.328684,19.0
46819,지에스25(GS25)삼성메디슨점,2018322016205600155,정상영업,GS25,37.507257,127.065775,LG25,2003377002405600057,정상영업,GS25,37.507252,127.065812,3.328684,19.0
46820,지에스25(GS25)삼성메디슨점,2018322016205600155,정상영업,GS25,37.507257,127.065775,LG25,2002377000505600039,정상영업,GS25,37.507252,127.065812,3.328684,19.0
37444,(주)씨스페이시스 대전구장점,2006365007905600008,정상영업,씨스페이스,36.325487,127.431148,(주)씨스페이시스 대흥상아점,2006365007905600009,정상영업,씨스페이스,36.325472,127.431130,2.262411,21.0
65060,바이더웨이 거평타운,1999322008305601332,정상영업,세븐일레븐,37.496980,127.030969,(주)바이더웨이 역삼타운점,2003322008305600270,정상영업,세븐일레븐,37.496989,127.030917,4.630674,23.0


## 같은 회사 근접편의점 수: 283->285

In [27]:
cvs_set_same = set()
cvs_set_same.update(lt_150.loc[idx_same_corp, 'src'].tolist())
cvs_set_same.update(lt_150.loc[idx_same_corp, 'dst'].tolist())

print(len(cvs_set_same))

408


In [38]:
idx_same = cvs['관리번호'].isin(cvs_set_same)
print(len(cvs.loc[idx_same, cols_cvs]))
cols_naver = cols_cvs.append('naver_distance')
cvs.loc[idx_same, cols_cvs].head(10)

# export
# cvs.loc[idx_same, cols_cvs].to_csv('same_damaged_cvs.tsv', sep='\t', index=False)

408


/home/jake/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,type,지역,naver_distance,naver_distance
1059,세븐일레븐 원효점,2015302015005600080,37.5313850196091,126.95534709344443,정상영업,"서울특별시 용산구 이촌로 1, 1층 102호 (한강로3가, GS한강에클라트)",서울특별시 용산구 한강로3가 16번지 85호,세븐일레븐,서울,NaN,NaN
1269,씨유 성수점,2008303010305600097,37.54462911301942,127.05467293539925,정상영업,서울특별시 성동구 아차산로 96 (성수동2가),서울특별시 성동구 성수동2가 314번지 6호,CU,서울,NaN,NaN
1294,GS25 상왕십리역,2018303009105600061,37.56422229114892,127.02902105215682,정상영업,"서울특별시 성동구 왕십리로 369, 동인 레반트 오피스텔 지하1층 B104호 (하왕...",서울특별시 성동구 하왕십리동 903번지 3호 동인 레반트 오피스텔,GS25,서울,NaN,NaN
1361,지에스25 성동체육센터,2015303009105600027,37.545840900720265,127.04402783824956,정상영업,서울특별시 성동구 왕십리로 89 (성수동1가),서울특별시 성동구 성수동1가 685번지 697호,GS25,서울,NaN,NaN
1403,GS25 성동서울숲점,2017303009105600037,37.546515140513456,127.04323611587547,정상영업,서울특별시 성동구 서울숲2길 41 (성수동1가),서울특별시 성동구 성수동1가 668번지 62호,GS25,서울,NaN,NaN
1425,지에스25S2 상왕십리역점,2017303009105600020,37.564467359880524,127.0290302021847,정상영업,"서울특별시 성동구 왕십리로 지하 374, 207-02호 (하왕십리동, 상왕십리역)",서울특별시 성동구 하왕십리동 946번지 14호,GS25,서울,NaN,NaN
1616,GS25 노유점,2007304010305600153,37.540142590888166,127.0677045629902,정상영업,서울특별시 광진구 동일로20길 78 (자양동),서울특별시 광진구 자양4동 6번지 39호,GS25,서울,NaN,NaN
1627,세븐일레븐 건대중앙점,2005304007305600117,37.542529826973656,127.07146342812048,정상영업,서울특별시 광진구 능동로13길 6 (화양동),서울특별시 광진구 화양동 3번지 42호,세븐일레븐,서울,NaN,NaN
1671,GS25 S군자역점,2016304016805600056,37.5572037603524,127.07949029946828,정상영업,"서울특별시 광진구 천호대로 지하 550, 군자역 지하1층 (능동)",서울특별시 광진구 능동 275번지 5호,GS25,서울,NaN,NaN
1704,CU 광진웰츠점,2017304019005600077,37.538247337794886,127.08385123668683,정상영업,"서울특별시 광진구 자양로 116, 116호 (구의동, 웰츠타워)",NaN,CU,서울,NaN,NaN


## Groupby: type

In [29]:
tmp = cvs.loc[idx_same, cols_cvs].groupby('type').agg({'관리번호':'count'})
print(tmp.sum())
print(tmp.sort_values('관리번호', ascending=False))

# tmp.reset_index().to_csv('same_near_cvs.tsv', sep='\t', index=False)

del tmp

관리번호    408
dtype: int64
       관리번호
type       
GS25    201
CU       97
세븐일레븐    76
미니스톱     32
씨스페이스     2


/home/jake/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


# 가장 가까운 편의점

In [30]:
lt_150.sort_values('naver_distance', ascending=True).head(20)

,src_name,src,src_status,src_type,src_lat,src_lng,dst_name,dst,dst_status,dst_type,dst_lat,dst_lng,line_distance,naver_distance
8609,미니스톱 역삼만남점,2010322016205600015,정상영업,미니스톱,37.495580,127.031844,미니스톱,2007324013905600118,정상영업,미니스톱,37.495578,127.031850,0.566976,12.0
8613,미니스톱 역삼만남점,2010322016205600015,정상영업,미니스톱,37.495580,127.031844,미니스톱,2009399021405600024,정상영업,미니스톱,37.495578,127.031850,0.566976,12.0
8611,미니스톱 역삼만남점,2010322016205600015,정상영업,미니스톱,37.495580,127.031844,미니스톱,1997362001205600154,정상영업,미니스톱,37.495578,127.031850,0.566976,12.0
64998,바이더웨이무역센터,2000322008305615906,정상영업,세븐일레븐,37.502014,127.057922,세븐일레븐 대치마루점,2009322012705600079,정상영업,세븐일레븐,37.502003,127.057917,1.266743,13.0
24741,세븐일레븐 과천원문점,2009397003205600008,정상영업,세븐일레븐,37.421158,126.991165,GS25 과천래미안점,2017397003205600013,정상영업,GS25,37.421185,126.991242,7.507307,13.0
66202,훼미리마트 뉴에버빌점,2008419015305600080,정상영업,CU,37.323882,127.957495,(주)코리아세븐 단관에버빌점,2012419020305600083,정상영업,세븐일레븐,37.323986,127.957577,13.660959,14.0
64645,훼미리마트성수점,200030300740560구16,정상영업,CU,37.544667,127.054900,씨유 성수점,2008303010305600097,정상영업,CU,37.544629,127.054673,20.421210,17.0
28120,씨유 횡계로타리점,2003428003905600016,정상영업,CU,37.672550,128.707489,GS25횡계점,2010428008305600025,정상영업,GS25,37.672292,128.707317,32.452135,17.0
2212,세븐일레븐사가정중앙점,2007306009505600030,정상영업,세븐일레븐,37.582154,127.087858,GS25 면목점,2017306014405600069,정상영업,GS25,37.582177,127.088153,26.127576,18.0
46819,지에스25(GS25)삼성메디슨점,2018322016205600155,정상영업,GS25,37.507257,127.065775,LG25,2003377002405600057,정상영업,GS25,37.507252,127.065812,3.328684,19.0


# 지역별 분석

## Index: 지역분류

In [31]:
regions = ['서울', '부산', '대구', '인천', '울산', '광주광역', '대전', '제주', '세종', '경기', '강원', '충청북도', '충북', '충청남도', '충남',
           '전라남도', '전남', '전라북도', '전북', '경상북도', '경북', '경상남도', '경남']

for idx, row in cvs.iterrows():
    road_addr = row['도로명전체주소']
    old_addr = row['소재지전체주소']
    
    for r in regions:
        try:
            if r in road_addr:
                cvs.at[idx, '지역'] = r
                break
            
        except TypeError:
            try:
                if r in old_addr:
                    cvs.at[idx, '지역'] = r
                    break
            except TypeError:
                pass
#             print(road_addr)

# cols_damaged = [*cols_cvs, '지역']

cvs.loc[cvs['지역']=='충청북도']

cvs.loc[idx_damaged, cols_cvs].head(1)

,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,type,지역
17,씨유 종로성대점,2010300012905600034,37.58443693473411,126.99702012247322,정상영업,서울특별시 종로구 성균관로5길 6 (명륜2가),서울특별시 종로구 명륜동2가 137번지 2호,CU,서울


## Group 'cvs' by '지역' 

In [32]:
# cols_cvs
print(len(cvs.loc[cvs['상세영업상태명']=='정상영업', cols_cvs]))
cvs_groupby_region = cvs.loc[cvs['상세영업상태명']=='정상영업', cols_cvs].groupby('지역').agg({'관리번호': 'count'})
print(cvs_groupby_region.sum())

cvs_groupby_region.sort_values('관리번호', ascending=False)

41252
관리번호    41252
dtype: int64


,관리번호
지역,
경기,10166
서울,8189
경상남도,2829
부산,2413
인천,2180
충청남도,2109
경상북도,1928
강원,1698
충청북도,1468


## Export

In [33]:
idx_specify = cvs['지역']=='강원'
print(len(cvs.loc[idx_damaged]))
cvs.loc[idx_damaged & idx_specify, cols_cvs]
# cvs.loc[(cvs['도로명전체주소'].str.contains('광주')) & (cvs['상세영업상태명']=='정상영업'), cols_damaged]

# Export
# cvs.loc[idx_damaged, cols_cvs].to_csv('damaged_cvs.tsv', sep='\t', index=False)
# cvs.loc[idx_damaged_100, cols_cvs].to_csv('damaged_cvs_100.tsv', sep='\t', index=False)


3011


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,type,지역
41694,씨유 춘천넥서스점,2011418016605600087,37.85234553188601,127.74519741400366,정상영업,"강원도 춘천시 퇴계농공로 9, 104호 (석사동,넥서스프라자빌딩)",강원도 춘천시 석사동 885번지 3호 넥서스프라자빌딩 104호,CU,강원
41702,GS25 춘천대학로점,2006418015505600077,37.87239325123567,127.74117138146121,정상영업,강원도 춘천시 백령로 115 (효자동),강원도 춘천시 효자동 769번지 15호,GS25,강원
41717,GS25 봄내박물관점,2011418016605600069,37.863120711424706,127.75086516458958,정상영업,"강원도 춘천시 우석로 46 (석사동,1층)",강원도 춘천시 석사동 764번지 1호 1층,GS25,강원
41751,씨유남춘천신시가지점,2013418022905600113,37.863344461062134,127.73134859479482,정상영업,"강원도 춘천시 남춘로 26, 1-3호 (퇴계동)",NaN,CU,강원
41752,세븐일레븐 춘천요선점,2014418022905600005,37.88185891265011,127.72809885770393,정상영업,강원도 춘천시 소양고개길 60 (요선동),NaN,세븐일레븐,강원
41757,미니스톱춘천석사점,2005418001605600088,37.851432053575365,127.74490189206686,정상영업,"강원도 춘천시 퇴계농공로 19 (석사동,화인프라자 1층)",강원도 춘천시 석사동 888번지 3 호 화인프라자 1층,미니스톱,강원
41802,미니스톱 춘천우석점,2017418027705600070,37.863285042073095,127.7505259118385,정상영업,강원도 춘천시 우석로 43 (석사동),NaN,미니스톱,강원
41819,씨유(CU)춘천대학로점,2015418027705600135,37.872106631260245,127.74090483341085,정상영업,강원도 춘천시 백령로 111 (효자동),NaN,CU,강원
41834,씨유 춘천요선점,2003418001605600053,37.88142336121095,127.72743529481147,정상영업,강원도 춘천시 서부대성로48번길 22 (요선동),강원도 춘천시 요선동 14번지 6 호,CU,강원
41845,이마트24 춘천강북점,2017418027705600031,37.89930902618172,127.72911208321574,정상영업,"강원도 춘천시 충열로16번길 15, 1층 (우두동)",NaN,이마트24,강원


## Validation: 지역&피해

In [34]:
number = '2017419024105600091'
idx_choose_cvs = (deduped['src']==number) | (deduped['dst']==number)
deduped.loc[idx_active & idx_choose_cvs, cols_distance]

# del tmp

,src_name,src,src_status,src_type,src_lat,src_lng,dst_name,dst,dst_status,dst_type,dst_lat,dst_lng,line_distance,naver_distance
27630,주식회사 이마트24 원주시청로점,2017419024105600091,정상영업,이마트24,37.340136,127.922175,지에스25영서본부점,2018419024105600061,정상영업,GS25,37.340449,127.92307,86.481865,83.0


In [35]:
tmp = cvs.loc[idx_damaged & cvs['지역'].isna(), cols_cvs]
print(len(tmp))
tmp.head()

0


,사업장명,관리번호,lat,lng,상세영업상태명,도로명전체주소,소재지전체주소,type,지역


## Groupby: 지역 & Export

In [36]:
tmp = cvs.loc[idx_damaged].groupby('지역').agg({'관리번호': 'count'})
print(tmp.sum())
tmp = tmp.rename(columns={'관리번호':'편의점 수'})

tmp.sort_values('편의점 수', ascending=False)

# Export
# tmp.sort_values('편의점 수', ascending=False).reset_index().to_csv('damaged_cvs_groupby_region.tsv', sep='\t', index=False)

관리번호    3011
dtype: int64


,편의점 수
지역,
서울,907
경기,737
부산,204
인천,181
경상남도,179
광주광역,96
강원,95
대전,85
대구,79
